In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, Input
from keras.models import Model

In [2]:
# Set the base directory
BASE_DIR = "UTKFace"

In [3]:
# Initialize lists for image paths and labels
image_paths = []
gender_labels = []

# Loop through each file in the dataset directory
for filename in tqdm(os.listdir(BASE_DIR)):
    image_path = os.path.join(BASE_DIR, filename)
    temp = filename.split('_')
    gender = int(temp[1])  # Extract only the gender
    image_paths.append(image_path)
    gender_labels.append(gender)

  0%|          | 0/23708 [00:00<?, ?it/s]

In [4]:
# Create a DataFrame
data = pd.DataFrame()
data['image'], data['gender'] = image_paths, gender_labels

# Function to extract features from images
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')
        img = img.resize((128,128))
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 128, 128, 1)
    return features

In [5]:
# Extract features and prepare labels
X = extract_features(data["image"])
y_gender = np.array(data["gender"])

  0%|          | 0/23708 [00:00<?, ?it/s]

In [6]:
# Define model input shape
input_shape = (128, 128, 1)

In [7]:
# Build the model
inputs = Input(input_shape)
conv_1 = Conv2D(32, kernel_size=(3,3), activation='relu')(inputs)
max_pool1 = MaxPooling2D(pool_size=(2,2))(conv_1)

conv_2 = Conv2D(64, kernel_size=(3,3), activation='relu')(max_pool1)
max_pool2 = MaxPooling2D(pool_size=(3,3))(conv_2)

conv_3 = Conv2D(128, kernel_size=(3,3), activation='relu')(max_pool2)
max_pool3 = MaxPooling2D(pool_size=(2,2))(conv_3)

conv_4 = Conv2D(256, kernel_size=(3,3), activation='relu')(max_pool3)
max_pool4 = MaxPooling2D(pool_size=(2,2))(conv_4)

flatten = Flatten()(max_pool4)
dense = Dense(256, activation='relu')(flatten)
dropout = Dropout(0.3)(dense)

In [8]:
# Final output layer for gender prediction
output = Dense(1, activation='sigmoid', name='gender_out')(dropout)

In [9]:
# Compile the model for gender prediction only
model = Model(inputs=inputs, outputs=output)
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Display model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128, 128, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 20, 20, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 18, 18, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 9, 9, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 7, 7, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 3, 3, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gender_out (Dense)                   │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 978,177 (3.73 MB)

 Trainable params: 978,177 (3.73 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train the model
history = model.fit(x=X, y=y_gender, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 274s 456ms/step - accuracy: 0.7006 - loss: 1.4256 - val_accuracy: 0.7893 - val_loss: 0.4599
Epoch 2/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 294s 496ms/step - accuracy: 0.8523 - loss: 0.3339 - val_accuracy: 0.7893 - val_loss: 0.4684
Epoch 3/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 237s 400ms/step - accuracy: 0.8628 - loss: 0.2959 - val_accuracy: 0.8075 - val_loss: 0.4526
Epoch 4/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 240s 404ms/step - accuracy: 0.8752 - loss: 0.2853 - val_accuracy: 0.8144 - val_loss: 0.4689
Epoch 5/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 230s 388ms/step - accuracy: 0.8893 - loss: 0.2521 - val_accuracy: 0.8007 - val_loss: 0.4405
Epoch 6/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 224s 378ms/step - accuracy: 0.8953 - loss: 0.2395 - val_accuracy: 0.8026 - val_loss: 0.4375
Epoch 7/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 222s 375ms/step - accuracy: 0.9059 - loss: 0.2282 - val_accuracy: 0.8134 - val_loss: 0.4566
Epoch 8/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 223s 376ms/step - accuracy: 0.9054 -

In [11]:
# Save the model
model.save("gender_classification_model4.h5")
